In [825]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
import scipy as sp
import sympy as sym


%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.4f}'.format})
GHz = 1e9

In [1182]:
fluxonium = sq_ext.sq_fluxonium()
H_0_f = fluxonium.hamiltonian() 
H_0 = qt.tensor( H_0_f, qt.qeye(H_0_f.shape[0]) ) + qt.tensor(qt.qeye(H_0_f.shape[0]), H_0_f ) #+ 0.00001*qt.tensor(fluxonium.flux_op(0), fluxonium.flux_op(0))
H = qt.tensor( H_0_f, qt.qeye(H_0_f.shape[0]) ) + qt.tensor(qt.qeye(H_0_f.shape[0]), H_0_f  )+ 1e5*qt.tensor(fluxonium.flux_op(0), fluxonium.flux_op(0))

In [1266]:
E, ψ_0_f = np.linalg.eigh(H_0_f.__array__() )
# E, ψ_0_f = sp.sparse.linalg.eigsh(H_0_f.data, 2, which='SA')
E, ψ_0_f = sq_ext.eigs_sorted(E, ψ_0_f)
ψ_0 = np.array([np.kron(ψ_0_f[:,0],ψ_0_f[:,0]), np.kron(ψ_0_f[:,1],ψ_0_f[:,0]), np.kron(ψ_0_f[:,0],ψ_0_f[:,1]), np.kron(ψ_0_f[:,1],ψ_0_f[:,1]) ]).T

In [1219]:
E, ψ_0 = sp.sparse.linalg.eigsh(H_0.data, 2, which='SA')
E, ψ_0 = sq_ext.eigs_sorted(E, ψ_0)
ψ_0 = np.linalg.qr(ψ_0)[0]

In [1256]:
E, ψ_0 = np.linalg.eigh(H_0.__array__())
ψ_0 =  ψ_0[:,:4]

In [1262]:
np.round(ψ_0.T.conj() @ ψ_0,4)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.-0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.-0.j, 0.-0.j, 1.+0.j, 0.+0.j],
       [0.-0.j, 0.-0.j, 0.-0.j, 1.+0.j]])

In [1267]:
H_eff = ψ_0.T.conj() @ H.__array__() @ ψ_0 / 2 / np.pi / GHz
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

 (15.3596+0j)	*	 I ⨂I 
 (-0.9727+0j)	*	 I ⨂σz 
 (0.0235+0j)	*	 σx ⨂σx 
 (-0.9727+0j)	*	 σz ⨂I 


In [1161]:
H_eff = sq_ext.H_eff_p1(H_0, H, out= 'GHz', n_eig=4, solver='numpy')
# H_eff = sq_ext.H_eff_p1(H_0, H, out= 'GHz', n_eig=4)
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

 (15.3596+0j)	*	 I ⨂I 
 (-0.961+0j)	*	 I ⨂σz 
 (0.0125+0j)	*	 σx ⨂σx 
 (-0.011+0j)	*	 σy ⨂σy 
 (-0.9845+0j)	*	 σz ⨂I 


In [1106]:
ψ_0[:,2] /= np.exp(1j*0.1)
H_eff = ψ_0.conj().T @ H.__array__() @ ψ_0 / 2 / np.pi / GHz
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

ValueError: shapes (4,4) and (625,625) not aligned: 4 (dim 1) != 625 (dim 0)

In [ ]:
def real_flux_basis_single(ψ_0,φ_ext , α, β, nmax, EJ, EC,μ=0):
    # The derivative of the energy w.r.t. the flux is 
    # flux operator because it act so on the +/- superposition states of currents
    e=1e-5
    Hf_plus = FQ_single( ϕ+e*2*np.pi, α, β, nmax,0,μ).Hamiltonian(EC=EC,EJ=EJ)
    Hf_minus= FQ_single( ϕ-e*2*np.pi, α, β, nmax,0,μ).Hamiltonian(EC=EC,EJ=EJ)
    F = (Hf_plus - Hf_minus)/(2*e*2*np.pi)
    
    #Substraction of the phase so <0|F|1> is real.
    Phase=ψb0[:,0].T.conj() @ F @ ψb0[:,1]/np.abs(ψb0[:,0].T.conj() @ F @ ψb0[:,1])
    ψb0_new=ψb0
    ψb0_new[:,1]=ψb0[:,1]/Phase